# Analyzing the similarity of major German Cities

## Introduction

Berlin and Hamburg are two of Germany's largest cities. In this analysis I will explore how similar or dissimilar they are based on Foursquare location data.

My target audience are people moving from one city to another and wanting to know where they should rent their new apartment.

Specifically, if you move from Hamburg to Berlin which neighborhood should you move to based on your previous neighborhood?

What characteristics do these neighborhoods have?

## Data

For each Hamburg neighborhood I will create a list of recommended Berlin neighborhoods based on how similar the mix of venue types is. 

I decided to use a regularly spaced grid of locations, centered around each city center, to define the neighborhoods.

The foollowing data sources will be needed to extract/generate the required information:

* The neighborhoods will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding
* The number of venues and their type and location in every neighborhood will be obtained using the Foursquare API.
* Geopy Nominatim will be used to obtain the city centers, using the Außenalster for Hamburg and the Brandenburg Gate for Berlin.

In [1]:
import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library

import re # for regular expressions

# for transforming geocoordinates
import shapely.geometry
import pyproj
import math

print('Libraries imported.')

Libraries imported.


### Generate neighborhoods

In [4]:
address = 'Brandenburg Gate, Berlin, Germany'

geolocator = Nominatim(user_agent="hamburg_explorer")
location = geolocator.geocode(address)
berlin_lat = location.latitude
berlin_lon = location.longitude
print('The geograpical coordinates of Berlin are {}, {}.'.format(berlin_lat, berlin_lon))

The geograpical coordinates of Berlin are 52.51628045, 13.37770188288172.


In [5]:
address = 'Außenalster, Hamburg, Germany'

geolocator = Nominatim(user_agent="hamburg_explorer")
location = geolocator.geocode(address)
hamburg_lat = location.latitude
hamburg_lon = location.longitude
print('The geograpical coordinates of Hamburg are {}, {}.'.format(hamburg_lat, hamburg_lon))

The geograpical coordinates of Hamburg are 53.5689488, 10.007305547125247.


In [8]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Hamburg center longitude={}, latitude={}'.format(hamburg_lon, hamburg_lat))
x, y = lonlat_to_xy(hamburg_lon, hamburg_lat)
print('Hamburg center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Hamburg center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Hamburg center longitude=10.007305547125247, latitude=53.5689488
Hamburg center UTM X=169483.03662988317, Y=5947163.190106782
Hamburg center longitude=10.007305547125249, latitude=53.568948799999994


In [179]:
berlin_center_x, berlin_center_y = lonlat_to_xy(berlin_lon, berlin_lat) # City center in Cartesian coordinates
hamburg_center_x, hamburg_center_y = lonlat_to_xy(hamburg_lon, hamburg_lat)

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
square_width = 10000
neigborhood_radius = 1500
x_step = neigborhood_radius
y_step = neigborhood_radius * k

x_min = berlin_center_x - square_width/2
y_min = berlin_center_y - square_width/2 - (int(21/k)*k*neigborhood_radius - square_width)/2
berlin_latitudes = []
berlin_longitudes = []
berlin_distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = neigborhood_radius/2 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        berlin_distance_from_center = calc_xy_distance(berlin_center_x, berlin_center_y, x, y)
        if (berlin_distance_from_center <= square_width/2+1):
            lon, lat = xy_to_lonlat(x, y)
            berlin_latitudes.append(lat)
            berlin_longitudes.append(lon)
            berlin_distances_from_center.append(berlin_distance_from_center)
            xs.append(x)
            ys.append(y)
            
x_min = hamburg_center_x - square_width/2
y_min = hamburg_center_y - square_width/2 - (int(21/k)*k*neigborhood_radius - square_width)/2
hamburg_latitudes = []
hamburg_longitudes = []
hamburg_distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = neigborhood_radius/2 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        hamburg_distance_from_center = calc_xy_distance(hamburg_center_x, hamburg_center_y, x, y)
        if (hamburg_distance_from_center <= square_width/2+1):
            lon, lat = xy_to_lonlat(x, y)
            hamburg_latitudes.append(lat)
            hamburg_longitudes.append(lon)
            hamburg_distances_from_center.append(hamburg_distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(berlin_latitudes), 'Berlin neighborhood centers generated.')
print(len(hamburg_latitudes), 'Hamburg neighborhood centers generated.')

39 Berlin neighborhood centers generated.
39 Hamburg neighborhood centers generated.


In [249]:
map_berlin = folium.Map(location=[berlin_lat, berlin_lon], zoom_start=12)

# add markers to map
for lat, lng in zip(berlin_latitudes, berlin_longitudes):
    label = '{}, {}'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=neigborhood_radius/40,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

In [181]:
map_hamburg = folium.Map(location=[hamburg_lat, hamburg_lon], zoom_start=12)

# add markers to map
for lat, lng in zip(hamburg_latitudes, hamburg_longitudes):
    label = '{}, {}'.format(lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=neigborhood_radius/40,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_hamburg)  
    
map_hamburg

In [217]:
hamburg_neighborhoods = []

for i in range(0,len(hamburg_latitudes)):
    reverse = geolocator.reverse((hamburg_latitudes[i],hamburg_longitudes[i]))
    address = reverse[0] 
    address_n = re.findall(".*, (.*),.*,.*,.*", address)[0]
    geo_lat = reverse[1][0]
    geo_lon = reverse[1][1]
    city = "Hamburg"
    hamburg_neighborhoods.append([address, geo_lat, geo_lon, address_n, city])

hamburg_neighborhoods = pd.DataFrame(hamburg_neighborhoods)
hamburg_neighborhoods.rename(columns={0:"Address",1:"Latitude",2:"Longitude",3:"Neighborhood",4:"City"}, inplace=True)

In [218]:
berlin_neighborhoods = []

for i in range(0,len(berlin_latitudes)):
    reverse = geolocator.reverse((berlin_latitudes[i],berlin_longitudes[i]))
    address = reverse[0] 
    try:
        address_n = re.findall(".*, (.*),.*,.*,.*", address)[0]
    except:
        address_n = re.findall("(.*),.*,.*,.*", address)[0]
    
    geo_lat = reverse[1][0]
    geo_lon = reverse[1][1]
    city = "Berlin"
    berlin_neighborhoods.append([address, geo_lat, geo_lon, address_n, city])

berlin_neighborhoods = pd.DataFrame(berlin_neighborhoods)
berlin_neighborhoods.rename(columns={0:"Address",1:"Latitude",2:"Longitude",3:"Neighborhood",4:"City"}, inplace=True)

In [219]:
hamburg_neighborhoods.head()

,Address,Latitude,Longitude,Neighborhood,City
0,"Zweite Querkanalbrücke, Worthdamm, Steinwerder...",53.532424,9.981784,Steinwerder,Hamburg
1,"37, Chicagokai, Quartier Elbtorquartier, Hafen...",53.534379,10.002156,HafenCity,Hamburg
2,"12, Zweibrückenstraße, Quartier Elbbrücken, Ha...",53.534061,10.025496,HafenCity,Hamburg
3,"129e, Marckmannstraße, Rothenburgsort, Hamburg...",53.534455,10.048362,Hamburg-Mitte,Hamburg
4,"Am Altonaer Holzhafen, Altona-Altstadt, Altona...",53.544220,9.945906,Altona,Hamburg


In [220]:
berlin_neighborhoods.head()

,Address,Latitude,Longitude,Neighborhood,City
0,"Pasta Bar, 1, Fritz-Reuter-Straße, Schöneberg,...",52.480966,13.349539,Tempelhof-Schöneberg,Berlin
1,"Rosenpromenade, KGA Papestraße, Tempelhof, Tem...",52.481159,13.371573,Tempelhof-Schöneberg,Berlin
2,"Flughafen Tempelhof, Werner-Loebermann-Weg, Ga...",52.480649,13.388919,Tempelhof-Schöneberg,Berlin
3,"Hasenschänke, Columbiadamm, Tempelhof, Tempelh...",52.482636,13.416267,Tempelhof-Schöneberg,Berlin
4,"24, Pommersche Straße, Wilmersdorf, Charlotten...",52.492595,13.315906,Wilmersdorf,Berlin


In [226]:
neighborhoods = pd.concat([hamburg_neighborhoods, berlin_neighborhoods])

In [227]:
hamburg_neighborhoods.to_csv("hamburg_neighborhoods.csv", index=False)
berlin_neighborhoods.to_csv("berlin_neighborhoods.csv", index=False)
neighborhoods.to_csv("neighborhoods.csv", index=False)

### Get venue data

In [228]:
limit = 20
radius = 1500 # see neighbourhood radius above

VERSION = '20180605' # Foursquare API version

In [229]:
%run credentials.py # client_id and client_secret for Foursquare

In [230]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [240]:
def getNearbyVenues(addresses, latitudes, longitudes, neighborhoods, cities, radius=500):
    
    venues_list=[]
    for add, lat, lng, neigh, city in zip(addresses, latitudes, longitudes, neighborhoods, cities):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            add,
            lat,
            lng,
            neigh,
            city,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Address', 
                  'Address Latitude', 
                  'Address Longitude',
                  'Neighborhood',
                  'City',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [242]:
hamburg_venues = getNearbyVenues(
    addresses = hamburg_neighborhoods['Address'],
    latitudes = hamburg_neighborhoods['Latitude'],
    longitudes = hamburg_neighborhoods['Longitude'],
    neighborhoods = hamburg_neighborhoods['Neighborhood'],
    cities = hamburg_neighborhoods['City']
)

In [244]:
berlin_venues = getNearbyVenues(
    addresses = berlin_neighborhoods['Address'],
    latitudes = berlin_neighborhoods['Latitude'],
    longitudes = berlin_neighborhoods['Longitude'],
    neighborhoods = berlin_neighborhoods['Neighborhood'],
    cities = berlin_neighborhoods['City']
)

In [253]:
all_venues = pd.concat([berlin_venues, hamburg_venues], ignore_index=True)

In [258]:
# exclude neighborhoods that are too small or too big
n_count = all_venues.groupby("Neighborhood").count()

n_incl = n_count[n_count["Venue"] >= 10].reset_index().Neighborhood

all_venues = all_venues[all_venues.Neighborhood.isin(n_incl)]

In [261]:
all_venues.head()

,Address,Address Latitude,Address Longitude,Neighborhood,City,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Pasta Bar, 1, Fritz-Reuter-Straße, Schöneberg,...",52.480966,13.349539,Tempelhof-Schöneberg,Berlin,Café de Enrico,52.481014,13.349788,Café
1,"Pasta Bar, 1, Fritz-Reuter-Straße, Schöneberg,...",52.480966,13.349539,Tempelhof-Schöneberg,Berlin,Osbili,52.479532,13.349973,Bistro
2,"Pasta Bar, 1, Fritz-Reuter-Straße, Schöneberg,...",52.480966,13.349539,Tempelhof-Schöneberg,Berlin,Sahara Sudanesische Spezialitäten,52.479845,13.351810,African Restaurant
3,"Pasta Bar, 1, Fritz-Reuter-Straße, Schöneberg,...",52.480966,13.349539,Tempelhof-Schöneberg,Berlin,Odeon,52.482086,13.349483,Indie Movie Theater
4,"Pasta Bar, 1, Fritz-Reuter-Straße, Schöneberg,...",52.480966,13.349539,Tempelhof-Schöneberg,Berlin,Balkan Grill,52.480042,13.352420,Eastern European Restaurant


In [263]:
all_venues.to_csv("all_venues.csv", index=False)

In [280]:
all_venues.groupby("Address").count().sort_values("Venue")

,Address Latitude,Address Longitude,Neighborhood,City,Venue,Venue Latitude,Venue Longitude,Venue Category
Address,,,,,,,,
"129e, Marckmannstraße, Rothenburgsort, Hamburg-Mitte, Hamburg, 20539, Deutschland",3,3,3,3,3,3,3,3
"Westhafenstraße, Moabit, Mitte, 13353, Deutschland",4,4,4,4,4,4,4,4
"Hammer Straße, Marienthal, Wandsbek, Hamburg, 22111, Deutschland",4,4,4,4,4,4,4,4
"12, Osterbrookplatz, Hamm, Hamburg-Mitte, Hamburg, 20537, Deutschland",4,4,4,4,4,4,4,4
"67c, Grandweg, Lokstedt, Eimsbüttel, Hamburg, 22529, Deutschland",5,5,5,5,5,5,5,5
"Mittelweg, Stellingen, Eimsbüttel, Hamburg, 22527, Deutschland",7,7,7,7,7,7,7,7
"24, Schlicksweg, Barmbek-Nord, Hamburg-Nord, Hamburg, 22307, Deutschland",7,7,7,7,7,7,7,7
"133, Süderstraße, Hammerbrook, Hamburg-Mitte, Hamburg, 20537, Deutschland",8,8,8,8,8,8,8,8
"Agentur für Arbeit Hamburg, Augustenburger Straße, Altona-Nord, Altona, Hamburg, 22769, Deutschland",8,8,8,8,8,8,8,8


### Find similarity between neighborhoods

In [276]:
venue_cat_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")
venue_cat_onehot['Neighborhood'] = all_venues['Neighborhood'] 
venue_cat_onehot['Address'] = all_venues['Address'] 

In [274]:
neighborhoods_grouped = venue_cat_onehot.groupby('Neighborhood').mean().reset_index()
neighborhoods_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Cafeteria,Café,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Office,Opera House,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,River,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Sushi Restaurant,Swabian Restaurant,Syrian Restaurant,Szechuan Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,Altona,0.0,0.000000,0.0,0.00000,0.00000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.00,0.062500,0.0,0.0,0.0,0.0,0.0,0.000000,0.020833,0.020833,0.0000,0.00000,0.0,0.0,0.000000,0.0,0.0,0.00000,0.020833,0.0,0.0,0.000000,0.00000,0.020833,0.0,0.083333,0.00,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.00000,0.0,0.020833,0.000000,0.000,0.000,0.00000,0.00000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00000,0.000,0.000000,0.000000,0.00000

In [277]:
address_grouped = venue_cat_onehot.groupby('Address').mean().reset_index()
address_grouped.head()

,Address,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Cafeteria,Café,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Capitol Building,Casino,Caucasian Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Lebanese Restaurant,Light Rail Station,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Office,Opera House,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Racetrack,Rafting,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,River,Rock Club,Roof Deck,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Sushi Restaurant,Swabian Restaurant,Syrian Restaurant,Szechuan Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,"10, Torstraße, Spandauer Vorstadt, Mitte, 1011...",0.0,0.0,0.0,0.0,0.05,0.0,0.00,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.050000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [278]:
address_grouped.shape

(75, 255)

## Methodology

## Results

## Discussion

## Conclusion